# Johnson-Cook with 6 strain rates

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("mime")
import pandas as pd
import h5py
from scipy.optimize import curve_fit
import lmfit

colors = ['#bb0000', '#00bb00', "#0000bb", '#bbbb00', '#bb00bb', "#00bbbb", '#bbbbbb', '#770000', '#007700', "#000077", '#777700', '#770077', "#007777", '#777777', '#440000', '#004400', "#000044", '#444400', '#440044', "#0044444", '#444444','#000000']

In [ ]:
baseSize = (8, 6)  # Base size of a subplot

def sbPlot(n):
    if (n == 1): return 1, 1
    if (n <= 2): return 1, 2
    if (n <= 4): return 2, 2
    if (n <= 6): return 3, 2
    if (n <= 9): return 3, 3
    if (n <= 12): return 4, 3
    return 0, 0

def sbPlotSize(n):
    x, y = sbPlot(n)
    return baseSize[0] * y, baseSize[1] * x

In [ ]:
h5f = h5py.File('../GleebleData.h5','r')
allData = h5f['all'][:]
shortData = h5f['short'][:]
h5f.close()

In [ ]:
allData.shape, shortData.shape

Remove first point of each curve, where $\varepsilon^p=0$

In [ ]:
allData = allData[allData[:,0] != 0]
shortData = shortData[shortData[:,0]!=0]
identData = shortData

In [ ]:
strains = np.unique(identData[:,0])
allStrains = np.unique(allData[:,0])
epsps = np.unique(identData[:,1])
temperatures = np.unique(identData[:,2])
nEps = len(strains)
nEpsp = len(epsps)
nTemp = len(temperatures)
strains, epsps, temperatures, nEps, nEpsp, nTemp

# Identification of the Johnson-Cook parameters
$\sigma = (A + B\varepsilon^n)\left(1 + C\ln\left(\frac{\dot{\varepsilon^p}}{\dot{\varepsilon^p}}\right)\right)\left(1-\left(\frac{T-T_0}{T_m - T_0}\right)^m\right)$

In [ ]:
T0 = temperatures[0]
epsp0 = epsps[0]
Tm = 1460

A is the initial yield at reference strain rate and temperature

In [ ]:
A = allData[(allData[:,1]==epsp0) & (allData[:,2]==T0)][0,3]
print("A=%g" %(A))

Extract the reference curve data

In [ ]:
Sig = identData[(identData[:,1]==epsp0) & (identData[:,2]==T0)]
params = np.polyfit(np.log(Sig[:,0]), np.log(Sig[:,3]-A), 1)
n, lnB = params
B = np.exp(lnB)
print("lnB=%g, B=%g, n=%g" %(lnB, B, n))

Extract data at reference temperature

In [ ]:
sigT0 = identData[identData[:,2]==T0]
sigT0[:,3] = ((sigT0[:,3])/(A + B*(sigT0[:,0])**n)) - 1

In [ ]:
def specialFunction(x,s):
    return s*x

S1 = []
cl=0
for epsj in list(strains):
    sbdata = sigT0[sigT0[:,0]==epsj]
    popt, pcov = curve_fit(specialFunction, np.log(sbdata[:,1]/epsp0), sbdata[:,3], bounds=(-np.inf, [np.inf]))
    s = popt
    S1.append(s)
    cl +=1
C  = np.mean(S1)
print("C=%g" %(C))

In [ ]:
sigEpsp0 = identData[(identData[:,1]==epsp0) & (identData[:,2]!=T0)]
sigEpsp0[:,3] = 1 - (sigEpsp0[:,3]/((A + B*(sigEpsp0[:,0])**n)))

In [ ]:
J1 = []
cl=0
for epsj in list(strains):
    sbdata = sigEpsp0[sigEpsp0[:,0]==epsj]
    #print(sbdata)
    popt, pcov = curve_fit(specialFunction, np.log((sbdata[:,2]-T0)/(Tm-T0)), np.log(sbdata[:,3]), bounds=(-np.inf, [np.inf]))
    m1 = popt
    J1.append(m1)
    cl +=1
m  = np.mean(J1)
print("m=%g" %(m))

In [ ]:
A,B,C,n,m

In [ ]:
def JCconstitutiveLaw(eps, epsp, T):
    return (A + B * eps**n) * (1 + C * np.log(epsp / epsp0)) * (1 - ((T - T0) / (Tm - T0))**m)

In [ ]:
# Plot the curves
from matplotlib.lines import Line2D
def create_dummy_line(**kwds):
    return Line2D([], [], **kwds)

plt.figure(figsize = sbPlotSize(nEpsp))
plt.rc('text', usetex = True)
idx = 1
plt.subplots_adjust(hspace = 0.3)
for epsp in list(epsps):
    xs, ys = sbPlot(nEpsp)
    plt.subplot(xs, ys, idx)
    sbdata = shortData[shortData[:,1]==epsp]
    cl =0
    for temp in list(temperatures):
        sbdata1 = sbdata[sbdata[:,2]==temp]
        plt.plot(sbdata1[:,0], sbdata1[:,3], colors[cl], marker = 's', markersize = 5, linestyle = 'none')
        plt.plot(strains, JCconstitutiveLaw(strains, epsp, temp), colors[cl], linewidth = 2.5)
        plt.rcParams['xtick.labelsize'] = 16
        plt.rcParams['ytick.labelsize'] = 16
        cl +=1
    plt.xlim(0, 0.7)
    plt.ylim(bottom=0)
    plt.xlabel(r'strain $\varepsilon$', fontsize = 16) # Labels the x axis
    plt.ylabel(r'flow stress $\sigma^y$ (MPa)', fontsize = 16) # Labels the y axis
    plt.title(r'strain rate $\dot{\varepsilon} = ' + str(epsp) + '$ s$^{-1}$', fontsize = 16) # Self explicit command
    idx += 1
    
legendLines = []
cl = 0
for temp in list(temperatures):
    legendLines.append((r'$T=$' + str(int(temp)) + r'$^{\circ}$C', {'color':colors[cl], 'linestyle':'-', 'linewidth':2.5, 'marker':'s'}))
    cl += 1

plt.legend([create_dummy_line(**l[1]) for l in legendLines],[l[0] for l in legendLines], 
           loc = 'upper center', fontsize = 12, ncols = 5, bbox_to_anchor = (0.0, -0.2), shadow = False)

plt.savefig("CompExpJC.svg")
plt.show()

In [ ]:
JCstress = JCconstitutiveLaw(allData[:,0], allData[:,1], allData[:,2])

In [ ]:
data = np.concatenate((allData[:,0:3],JCstress.reshape((JCstress.shape[0],1))),axis=1)
h5f = h5py.File('JC-6.h5','w')
h5f.create_dataset('data', data = data)
h5f.close()

In [ ]:
EAAR = np.sum(np.abs((allData[:,3] - JCstress)/(allData[:,3])))*100/JCstress.shape[0]
print("EAAR = %g" %(EAAR) + ' %')

In [ ]:
RMSE = np.sqrt(np.sum((allData[:,3] - JCstress)**2)/allData.shape[0])
print('RMSE = %g' %(RMSE)+' MPa')

In [ ]:
def outOf(val, i):
    if ((val<-i) or (val>i)): return True
    return False

def conv (v, d):
    va = abs(v)
    e = int(np.log10(va))
    if (va < 1): e-=1
    if outOf(e,2):
        a = v * 10**(-e)
        v = np.round(a * 10**d)/10**d
        return str(v)+'\\times 10^{'+str(e)+'}'
    v = np.round(v * 10**d)/10**d
    return str(v)

In [ ]:
print('A =',conv(A,4))
print('B =',conv(B,4))
print('n =',conv(n,4))
print('C =',conv(C,4))
print('m =',conv(m,4))